## Clustering for Codekicker.de

### Introduction

In the notebook, have used **Nltk** as the main library to solve the NLP task of clustering input sentences into 5 groups.

Different text preprocessing techniques such as tokenization, stemming, lemmatization, part-of-speech tagging are done using the nltk library which are specific to German language. 

A simple method of clustering sentences based on the **frequency counting of the words** that matches with the specific tags is performed in this notebook. 

In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import nltk
import re
import codecs

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.snowball import GermanStemmer
from string import punctuation
from nltk.corpus import stopwords
from nltk import pos_tag

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Created an instance of object specific to stemming and stopwords that supports German language
stemmer = GermanStemmer()
stops = set(stopwords.words('german'))

# Loading punctuations
punctuations = punctuation

Read the input text file from "unclustered_input.txt" using codecs module in order to deal with the encoding issues (German Umlauts)

In [3]:
# Loading dataset from "unclustered_input.txt"
data = codecs.open('unclustered_input.txt', 'r', 'utf-8')
data = data.read()
data

'Meine Email wird nicht versendet.\nWenn ich Email versende, kommt eine Fehlermeldung.\nIch kann Emails nicht mehr verschicken.\nOutlook funktioniert nicht mehr.\nThunderbird funktioniert nicht mehr.\nDas Versenden von Emails funktioniert nicht.\nDas Verschicken von Email funktioniert nicht.\nBeim Verschicken von Emails kommt eine Fehlermeldung.\nBeim Versenden von Emails kommt eine Fehlermeldung.\nIch kann keine Emails über Thunderbird versenden.\nEmails über Outlook zu versenden geht nicht.\nMeine Emailadresse funktioniert mehr.\nIch kann niemanden per Email erreichen.\nMein Emailprogramm funktioniert nicht mehr.\nDer Versendeladebalken stoppt.\nEine Fehlermeldung „Sie benötigen Adminrechte“ poppt auf.\nWenn ich das Programm installiere, kommt ein Fehler.\nDie Installation des Programms bricht mit einer Fehlermeldung ab.\nDie Installation des Programms startet nicht.\nIch klicke auf Setup-Datei und dann zeigt er mir einen Fehler.\nIch habe keine Admin-Rechte und kann deshalb nichts i

### Text Preprocessing and Clustering

In [4]:
# Performing sentence tokenize
sentences = sent_tokenize(data)
sentences

['Meine Email wird nicht versendet.',
 'Wenn ich Email versende, kommt eine Fehlermeldung.',
 'Ich kann Emails nicht mehr verschicken.',
 'Outlook funktioniert nicht mehr.',
 'Thunderbird funktioniert nicht mehr.',
 'Das Versenden von Emails funktioniert nicht.',
 'Das Verschicken von Email funktioniert nicht.',
 'Beim Verschicken von Emails kommt eine Fehlermeldung.',
 'Beim Versenden von Emails kommt eine Fehlermeldung.',
 'Ich kann keine Emails über Thunderbird versenden.',
 'Emails über Outlook zu versenden geht nicht.',
 'Meine Emailadresse funktioniert mehr.',
 'Ich kann niemanden per Email erreichen.',
 'Mein Emailprogramm funktioniert nicht mehr.',
 'Der Versendeladebalken stoppt.',
 'Eine Fehlermeldung „Sie benötigen Adminrechte“ poppt auf.',
 'Wenn ich das Programm installiere, kommt ein Fehler.',
 'Die Installation des Programms bricht mit einer Fehlermeldung ab.',
 'Die Installation des Programms startet nicht.',
 'Ich klicke auf Setup-Datei und dann zeigt er mir einen Fehl

In [5]:
# Initializing the list of tags to be considered to cluster the sentences
tags = [
    ['versenden', 'email', 'verschiken', 'outlook', 'thunderbird'],
    ['installieren', 'installation', 'admin', 'admin-rechte', 'setup'],
    ['maus', 'mauszeiger', 'zeiger', 'cursor', 'trackpad', 'mousepad'],
    ['installation', 'excel', 'powerpoint',
        'formattierung', 'computer', 'abstürzen']
]

# Stemming the words in tags to get the root word
tags = [[stemmer.stem(j.lower()) for j in i] for i in tags]

tags

[['versend', 'email', 'verschik', 'outlook', 'thunderbird'],
 ['installi', 'installation', 'admin', 'admin-recht', 'setup'],
 ['maus', 'mauszeig', 'zeig', 'cursor', 'trackpad', 'mousepad'],
 ['installation', 'excel', 'powerpoint', 'formattier', 'comput', 'absturz']]

In [6]:
tag_length = len(tags)

# Initializing a list of empty dictionary for each cluster to store the tag count
tag_count = [{} for i in range(tag_length)]

# Append an empty dictionary,since there are no specific tags for cluster 5
tag_count.append({})

# Initalizing the tag count to 0 for each cluster with the tags
for i in range(tag_length):
    for tag in tags[i]:
        tag_count[i][tag] = 0

Here I have looped through each sentences and done few text processing like word tokenization - where each sentence is 
tokenized as seperate words, checking if the tokens are punctautaions, checking if the tokens are present in the stopwords 
(German stop words), finally stemming the token to have its base word. 

And counting the occurance of tags in the sentences by looping through the tags. 

If a tag is found, then the cluster with the highest tag count is identified and the tag count is incremented in the cluster 
for each tag. 

If the sentence doesnot match any tags from the tag list then the sentence is assumed to be belonging to cluster 5 and is added 
to the tag count as 1.

In [7]:
# Processing input sentences and counting tags in each cluster
for sentence in sentences:

    # Initalizing list of zeros to store count of tags in the current sentence
    sent_count = [0 for i in range(tag_length)]

    # Boolean variable is initialised to check the presence of tags
    tags_ = False
    words = word_tokenize(sentence)
    for word in words:
        if word.lower() not in punctuations:
            if word.lower() not in stops:
                word = stemmer.stem(word.lower())

                # Looping over the tag cluster
                for i in range(len(tags)):

                    # Checking if the current word is in the current tag cluster
                    if word in tags[i]:

                        # Incrementing the count of the current tag cluster in the sentence couunt list
                        sent_count[i] += 1

                        # Setting the tags true,if the current sentence has atleast one tag
                        tags_ = True
                        break

    # Checking for tags
    if tags_:

        # Finding the maximum count of the tags in sentence count list
        max_count = max(sent_count)

        # Getting the index of tag count cluster with high sentence count list
        cluster_index = sent_count.index(max_count)
        for word in sentence.lower().split():
            word = stemmer.stem(word.lower())

            # Checking the word in the tag cluster with highest count
            if word in tags[cluster_index]:

                # Incrementing the count of the current word in the tag cluster with highest count
                tag_count[cluster_index][word] += 1
    else:

        # Adding the current sentence to the notags dict with the count of 1
        tag_count[4][sentence] = 1

Again looping through each sentences and checking for certain conditions, assigning each sentence to specific cluster with the highest tag counts. If the sentence do not match to any of the tags then it is assigned to cluster 5. 

An empty list is initialised in the begining where the sentences and its corresponding clusters are stored. 

In [8]:
# Assign each sentence to the cluster with the highest tag count
results = []
for sentence in sentences:
    sent_count = [0 for i in range(len(tags))]
    tags_ = False
    words = word_tokenize(sentence)
    for word in words:
        if word.lower() not in punctuations:
            if word.lower() not in stops:
                word = stemmer.stem(word.lower())
                for i in range(len(tags)):
                    if word in tags[i]:
                        sent_count[i] += 1
                        tags_ = True
                        break

    if tags_:
        max_count = max(sent_count)
        cluster_index = sent_count.index(max_count)

        # Checking if the highest tag count is equal to 0
        if max_count == 0:

            # If the count is zero assigning the sentence to no tags cluster
            cluster_index = 4
        results.append((sentence, cluster_index+1))
    else:
        results.append((sentence, 5))

In [9]:
# Creating function to view the sentences in each clusters
def cluster_sent(val):
    cluster = [i[0] for i in results if i[1] == val]
    print('cluster_', val)
    for i in cluster:
        print('\n', i)


cluster_sent(1)
cluster_sent(2)
cluster_sent(3)
cluster_sent(4)
cluster_sent(5)

cluster_ 1

 Meine Email wird nicht versendet.

 Wenn ich Email versende, kommt eine Fehlermeldung.

 Ich kann Emails nicht mehr verschicken.

 Outlook funktioniert nicht mehr.

 Thunderbird funktioniert nicht mehr.

 Das Versenden von Emails funktioniert nicht.

 Das Verschicken von Email funktioniert nicht.

 Beim Verschicken von Emails kommt eine Fehlermeldung.

 Beim Versenden von Emails kommt eine Fehlermeldung.

 Ich kann keine Emails über Thunderbird versenden.

 Emails über Outlook zu versenden geht nicht.

 Ich kann niemanden per Email erreichen.
cluster_ 2

 Wenn ich das Programm installiere, kommt ein Fehler.

 Die Installation des Programms bricht mit einer Fehlermeldung ab.

 Die Installation des Programms startet nicht.

 Ich habe keine Admin-Rechte und kann deshalb nichts installieren.

 Ich kann das Programm nicht installieren.

 Eine Fehlermeldung taucht beim Installieren auf.

 Ein Fehler erscheint bei der Installation.

 Ich brauche Hilfe bei der Installation von Exc

In [10]:
# Creating dataframe from the results
df = pd.DataFrame(results, columns=['Sentence', 'Cluster'])
df

,Sentence,Cluster
0,Meine Email wird nicht versendet.,1
1,"Wenn ich Email versende, kommt eine Fehlermeld...",1
2,Ich kann Emails nicht mehr verschicken.,1
3,Outlook funktioniert nicht mehr.,1
4,Thunderbird funktioniert nicht mehr.,1
5,Das Versenden von Emails funktioniert nicht.,1
6,Das Verschicken von Email funktioniert nicht.,1
7,Beim Verschicken von Emails kommt eine Fehlerm...,1
8,Beim Versenden von Emails kommt eine Fehlermel...,1
9,Ich kann keine Emails über Thunderbird versenden.,1


### Model Evaluation

In [11]:
# Creating dataframe with the test data with truth cluster labels
df1 = pd.read_table('cluster_1.txt', names=['Sentence', 'Cluster'])
df1['Cluster'] = 1
df2 = pd.read_table('cluster_2.txt', names=['Sentence', 'Cluster'])
df2['Cluster'] = 2
df3 = pd.read_table('cluster_3.txt', names=['Sentence', 'Cluster'])
df3['Cluster'] = 3
df4 = pd.read_table('cluster_4.txt', names=['Sentence', 'Cluster'])
df4['Cluster'] = 4
df5 = pd.read_table('cluster_5.txt', names=['Sentence', 'Cluster'])
df5['Cluster'] = 5

# Creating merged dataframe using pandas concat
merged_df = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)
merged_df

,Sentence,Cluster
0,Meine Email wird nicht versendet.,1
1,"Wenn ich Email versende, kommt eine Fehlermeld...",1
2,Ich kann Emails nicht mehr verschicken.,1
3,Outlook funktioniert nicht mehr.,1
4,Thunderbird funktioniert nicht mehr.,1
5,Das Versenden von Emails funktioniert nicht.,1
6,Das Verschicken von Email funktioniert nicht.,1
7,Beim Verschicken von Emails kommt eine Fehlerm...,1
8,Beim Versenden von Emails kommt eine Fehlermel...,1
9,Ich kann keine Emails über Thunderbird versenden.,1


In [12]:
# Assigning truth label and the predictred label from two datframes to the variables
y_true = merged_df['Cluster']
y_pred = df['Cluster']

# Assigning target cluster names
target_names = ['cluster_1', 'cluster_2',
                'cluster_3', 'cluster_4', 'cluster_5']

print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

   cluster_1       1.00      0.80      0.89        15
   cluster_2       0.88      0.78      0.82         9
   cluster_3       1.00      0.80      0.89        10
   cluster_4       0.67      0.67      0.67         3
   cluster_5       0.33      1.00      0.50         3

    accuracy                           0.80        40
   macro avg       0.78      0.81      0.75        40
weighted avg       0.90      0.80      0.83        40



### Model Evaluation with New Input

In [13]:
# Putting together the entire process inside a function
def text_clustering(sentences):
    tag_count = [{} for i in range(len(tags))]
    tag_count.append({})

    for i in range(tag_length):
        for tag in tags[i]:
            tag_count[i][tag] = 0

    for sentence in sentences:
        sent_count = [0 for i in range(tag_length)]
        tags_ = False
        words = word_tokenize(sentence)
        for word in words:
            if word.lower() not in punctuations:
                if word.lower() not in stops:
                    word = stemmer.stem(word.lower())
                    for i in range(len(tags)):
                        if word in tags[i]:
                            sent_count[i] += 1
                            tags_ = True
                            break

        if tags_:
            max_count = max(sent_count)
            cluster_index = sent_count.index(max_count)
            for word in sentence.lower().split():
                word = stemmer.stem(word.lower())
                if word in tags[cluster_index]:
                    tag_count[cluster_index][word] += 1
        else:
            tag_count[4][sentence] = 1

    results = []
    for sentence in sentences:
        sent_count = [0 for i in range(len(tags))]
        tags_ = False
        words = nltk.word_tokenize(sentence)
        for word in words:
            if word.lower() not in punctuations:
                if word.lower() not in stops:
                    word = stemmer.stem(word.lower())
                    for i in range(len(tags)):
                        if word in tags[i]:
                            sent_count[i] += 1
                            tags_ = True
                            break

        if tags_:
            max_count = max(sent_count)
            cluster_index = sent_count.index(max_count)
            if max_count == 0:
                cluster_index = 4
            results.append((sentence, cluster_index+1))
        else:
            results.append((sentence, 5))

    return results

In [14]:
# Checking the cluster with new input
text_clustering(['Ich kann nicht schwimmen'])

[('Ich kann nicht schwimmen', 5)]

In [15]:
text_clustering(
    ['Schwerwiegender Fehler: Maximale Ausführungszeit von 30 Sekunden überschritten in...(beim Installieren)'])

[('Schwerwiegender Fehler: Maximale Ausführungszeit von 30 Sekunden überschritten in...(beim Installieren)',
  2)]

In [16]:
text_clustering(['Mein Computer funktioniert nicht'])

[('Mein Computer funktioniert nicht', 4)]

In [17]:
text_clustering(['Ich erhalte die meisten Emails nicht'])

[('Ich erhalte die meisten Emails nicht', 1)]

In [18]:
text_clustering(['Ich habe meine Maus kaputt gemacht'])

[('Ich habe meine Maus kaputt gemacht', 3)]

### Inference

Using frequency counting with respect to tags is choosen to cluster the input sentences, as it performs better than the other models. It gives better precision and recall scores comparatively. 

It is also observed that text pre processing like stemming helped in improving the performance of the model, however using spacy lemmatization did not make any difference with the performance. So i went with stemming. Using pos_tag did not make any difference either.

The precision score of cluster 5 is less, may be because of the insuffiecient tags. However the Recall is good.
    
Having more tags(words) will help with better clustering. And having more datas could also help with the performance.

### Comparing with Other Models

Have tried different Nlp and Ml models like KMeans Clustering , Agglomerative Clustering, Spectral Clustering, and BERT. But the models did not perform better than this model.

Even after text pre processing, using dimensionality reduction techniques, and normalizing the data, the performance of the model did not improve exponentially. Different hyperparameters were also experimented with each clustering algorithms.

This could be because of not having enough dataset to train the model. By increasing the size of the dataset the model could have better understood the patterns in the data to perform better clustering.

Having an imbalance in the dataset, ie the cluster 4 and 5 have very less data comparatively. Imbalanced dataset could also be a reason to affect the performace in ML models.

### Future work

I could have done data augumentation to increase the number of data to achieve better performance.

I could have also tried different embedding techniques like word2vec, GloVe and FastText.

I could have also used ensemble methods.